<a href="https://colab.research.google.com/github/ludawg44/jigsawlabs/blob/master/28Apr20_3_coerce_dupes_and_dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identify DataTypes

### Introduction

We now have the task of working with the rest of our training data.  Now, when working with so many columns it's often difficult to know where to begin.  Our general technique will be to capture as much of our data as possible, and then feed it into our model to get some feedback on the model performance and feature importances.  

To do that, to capture as much data as possible, we effectively have to coerce as many columns as possible into a number.  We can accomplish this in stages, working with the data that is easiest to coerce first. Let's get started.

### Sizing it Up

> You can find this dataset [here](https://github.com/jigsawlabs-student/engineering-large-datasets/blob/master/X_train_listings.feather).  You need to download by clicking *view raw*, and then upload to your colab notebook.

In [0]:
import pandas as pd 

X_train_df = pd.read_csv('https://raw.githubusercontent.com/jigsawlabs-student/engineering-large-datasets/master/price_listings_ten_k.csv', index_col = 0)

Let's start by reducing our problem.  We have 96 columns to work through with our Airbnb data, but some of them are unnecessary.  The only columns that we can be confident truly add no benefit are those where each value in the column is identical.  

In [0]:
def find_all_same(df):
    return [col for col in df.columns if len(df[col].unique()) == 1]

In [0]:
same_cols = find_all_same(X_train_df)

same_cols

['scrape_id',
 'experiences_offered',
 'thumbnail_url',
 'medium_url',
 'xl_picture_url',
 'host_acceptance_rate',
 'country_code',
 'country',
 'has_availability',
 'jurisdiction_names',
 'is_business_travel_ready']

Let's confirm we did this correctly and take a look.

In [0]:
X_train_df[same_cols][:2]

,scrape_id,experiences_offered,thumbnail_url,medium_url,xl_picture_url,host_acceptance_rate,country_code,country,has_availability,jurisdiction_names,is_business_travel_ready
0,2.018111e+13,none,NaN,NaN,NaN,NaN,DE,Germany,t,NaN,f
1,2.018111e+13,none,NaN,NaN,NaN,NaN,DE,Germany,t,NaN,f


Ok great, let's get rid of this column.

In [0]:
X_train_df_var = X_train_df.drop(columns = same_cols)

In [0]:
X_train_df_var.shape

(8000, 83)

So right off the bat, we got rid of some columns.  Now let's further reduce our problem by finding those columns that are of type object.  These will be the ones that we'll need to work on coercing.

In [0]:
X_train_object_df = X_train_df_var.select_dtypes(include = 'object')

In [0]:
X_train_object_df.shape

(8000, 54)

So we see that there are 29 columns that we can effectively use and 56 that we cannot use.

### Coerce To DateTimes

Now let's identify and coerce our date columns.  We first use our method to identify the date columns.

In [0]:
def contains_date(column):
#     remove nas first, potentially use all
    regex_string = (r'^\d{1,2}-\d{1,2}-\d{4}$|^\d{4}-\d{1,2}-\d{1,2}$' + 
'|^\d{1,2}\/\d{1,2}\/\d{4}$|^\d{4}\/\d{1,2}\/\d{1,2}$')
    return column.str.contains(regex_string).any()

In [0]:
potential_date_cols = [col for col in X_train_object_df.columns if contains_date(X_train_object_df[col])]

In [0]:
potential_date_cols

['last_scraped',
 'host_since',
 'calendar_last_scraped',
 'first_review',
 'last_review']

Then we take a look.

In [0]:
almost_date_df = X_train_object_df[potential_date_cols]
almost_date_df[:2]

,last_scraped,host_since,calendar_last_scraped,first_review,last_review
0,2018-11-07,2008-08-18,2018-11-07,2016-04-11,2018-10-28
1,2018-11-07,2008-09-16,2018-11-07,2018-07-04,2018-10-01


We have identified five columns that we can set as datetimes.  Let's do so.

In [0]:

date_df = almost_date_df.apply(lambda col: pd.to_datetime(col, errors = 'coerce'))
date_df[:3]

,last_scraped,host_since,calendar_last_scraped,first_review,last_review
0,2018-11-07,2008-08-18,2018-11-07,2016-04-11,2018-10-28
1,2018-11-07,2008-09-16,2018-11-07,2018-07-04,2018-10-01
2,2018-11-07,2008-10-19,2018-11-07,2009-06-20,2017-03-20


Now we can replace our original dataframe with these columns.

In [0]:
X_train_df_var.loc[:, date_df.columns] = date_df

So we can see that we just coerced five more columns.

In [0]:
X_train_df_var.select_dtypes(include = 'object').shape

(8000, 49)

So we just finished with coercing to dates.  It wasn't so bad.  We just needed to identify the datecolumns, and pass a datatype dictionary to the `astype` method.

### Coercing Numbers

Let's move onto seeing if there are any columns that we can convert to numbers.  We start off in a similar way as before.  We again select the columns that are of type object, and then see which of those appear to contain numbers.

In [0]:
X_train_object_df = X_train_df_var.select_dtypes(include = 'object')

> Now that we narrowed down to object columns, we can see which ones contain numbers.

To do so, we have a function that looks through a column and determines if there are any numbers.

In [0]:
def contains_numbers(column):
    # matches price or percentage     
    regex_string = (r'^(?!.*www|.*-|.*\/|.*[A-Za-z]|.* ).*\d.*')
#     regex_string = (r'\$\d+.*|\d+.*\%$|^\d+.*$')
    return column.str.contains(regex_string).all()

> We can change the function to end with `any` instead of `all` to make it less restrictive.

Then, we loop through our columns to see which contain numbers.

In [0]:
potential_num_cols = [col for col in X_train_object_df.columns if contains_numbers(X_train_object_df[col])]

In [0]:
potential_num_cols

['host_response_rate', 'security_deposit', 'cleaning_fee', 'extra_people']

Let's take a look.

In [0]:
almost_num_df = X_train_object_df[potential_num_cols]
almost_num_df[:6]

,host_response_rate,security_deposit,cleaning_fee,extra_people
0,96%,$200.00,$30.00,$28.00
1,NaN,$0.00,$0.00,$0.00
2,100%,$200.00,$50.00,$20.00
3,NaN,$250.00,$30.00,$18.00
4,100%,$0.00,$0.00,$24.00
5,100%,$400.00,$80.00,$10.00


Ok, so our function did a pretty good job.

Ok now that we've identified our almost number columns, let's work on coercing them.  

We already have `remove_price` method, which seems like it will work on our `security_deposit`, `cleaning_fee` and `extra_people` columns.

> One thing to consider is whether these last three columns really belong in our features.  The columns  are related to how much our host is charging for the apartment, which is what we are trying to predict.  Let's include it for now, but this may be something to remove later on.

Ok, we can start by selecting a single column, and then working on converting it.  We'll start with the `host_response_rate`.

In [0]:
host_resp_rate = almost_num_df['host_response_rate']

In [0]:
host_resp_rate.loc[5]

'100%'

Then we can write a function that removes a percentage where one exists, simply converts null values to be numeric.

In [0]:
import pandas as pd
def convert_percent(val):
    if pd.isna(val):
        return pd.to_numeric(val, errors='coerce', downcast='float')
    else:
        without_dollar = val[:-1]
        return float(without_dollar)

We can test that this works properly with the map function.

In [0]:
host_resp_rate.map(convert_percent)[:3]

0     96.0
1      NaN
2    100.0
Name: host_response_rate, dtype: float64

And then we can use the function with a FunctionTransformer.

In [0]:
from sklearn_pandas import FunctionTransformer, DataFrameMapper

In [0]:
from sklearn.impute import SimpleImputer
mapper = DataFrameMapper([
    (['host_response_rate'], FunctionTransformer(convert_percent))
], df_out = True)

In [0]:
transformed_host_rate = mapper.fit_transform(almost_num_df[['host_response_rate']])

In [0]:
transformed_host_rate[:3]

,host_response_rate
0,96.0
1,NaN
2,100.0


Great, so now we wrote a DataFrameMapper to convert this into a numeric column.

In [0]:
transformed_host_rate.dtypes

host_response_rate    float64
dtype: object

Let's move onto the rest of the columns.

In [0]:
almost_num_df[:6]

,host_response_rate,security_deposit,cleaning_fee,extra_people
0,96%,$200.00,$30.00,$28.00
1,NaN,$0.00,$0.00,$0.00
2,100%,$200.00,$50.00,$20.00
3,NaN,$250.00,$30.00,$18.00
4,100%,$0.00,$0.00,$24.00
5,100%,$400.00,$80.00,$10.00


We'll convert the price columns in the same way. 

In [0]:
def convert_price(val):
    if pd.isnull(val):
        return pd.to_numeric(val) 
    else:
        without_commas = val.replace(',','')
        without_dollar = without_commas[1:]
        return pd.to_numeric(without_dollar)

> Then we can place this integrate this function into a step with a FunctionTransformer.

In [0]:
(['security_deposit'], [FunctionTransformer(convert_price)])

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


(['security_deposit'], [FunctionTransformer(func=None)])

But remember, we have three columns that we wish to convert.

In [0]:
potential_num_cols[1:]

['security_deposit', 'cleaning_fee', 'extra_people']

So we can loop through and create a list of the steps with code.

In [0]:
convert_dolls_to_nums = [([col], [FunctionTransformer(convert_price)]) 
                         for col in potential_num_cols[1:]]

In [0]:
convert_dolls_to_nums

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


[(['security_deposit'], [FunctionTransformer(func=None)]),
 (['cleaning_fee'], [FunctionTransformer(func=None)]),
 (['extra_people'], [FunctionTransformer(func=None)])]

Take a look at the code above.  We'll be doing this a lot.  All we're doing is looping through to create multiple steps with list iteration.  So we specify the each step that we want to return as:
```python
([col], [FunctionTransformer(convert_price)])
```

Once our steps are defined, we integrate them in our DataFrameMapper.

In [0]:
from sklearn.impute import SimpleImputer
convert_percent_to_nums = [ (['host_response_rate'], [FunctionTransformer(convert_percent)]) ]
convert_dolls_to_nums
convert_to_nums_steps = convert_percent_to_nums + convert_dolls_to_nums
to_number_mapper = DataFrameMapper(convert_to_nums_steps, df_out = True, default = None)

In [0]:
converted_nums_df = to_number_mapper.fit_transform(X_train_df_var)

converted_nums_df[:1]

,host_response_rate,security_deposit,cleaning_fee,extra_people,id,listing_url,last_scraped,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,...,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,guests_included,minimum_nights,maximum_nights,calendar_updated,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,96.0,200.0,30.0,28.0,2015,https://www.airbnb.com/rooms/2015,2018-11-07 00:00:00,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,It is located in the former East Berlin area o...,"This is my home, not a hotel. I rent out occas...","Close to U-Bahn U8 and U2 (metro), Trams M12, ...","Simple kitchen/cooking, refrigerator, microwav...",Always available,"No parties No events No pets No smoking, not e...",https://a0.muscache.com/im/pictures/260fd609-7...,2217,https://www.airbnb.com/users/show/2217,Ian,2008-08-18 00:00:00,"Key Biscayne, Florida, United States",Believe in sharing economy.,within an hour,t,https://a0.muscache.com/im/pictures/21428a22-4...,https://a0.muscache.com/im/pictures/21428a22-4...,Mitte,4,4,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Berlin, Berlin, Germany",Mitte,Brunnenstr. Süd,Mitte,Berlin,Berlin,...,52.5345,13.4026,f,Guesthouse,Entire home/apt,3,1,1,2,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil...",NaN,60,1,4,1125,3 months ago,0,21,51,141,2018-11-07 00:00:00,118,2016-04-11 00:00:00,2018-10-28 00:00:00,93,10,9,10,10,10,9,t,NaN,f,strict_14_with_grace_period,f,f,4,3.76


So things look pretty good.  We added default = None, so that we return the entire dataframe.

Unfortunately, the `default = None`, changed our unspecified columns to be type Object.  No worry, we change them back.  We do so by collecting the dtypes from our dataframe above.  And then only selecting those datatypes that were of type object.

In [0]:
dtypes = dict(X_train_df_var.dtypes)

In [0]:
dtypes = {k:v for k, v in dtypes.items() if v.kind is not 'O'}

In [0]:
list(dtypes.items())[:5]

[('id', dtype('int64')),
 ('last_scraped', dtype('<M8[ns]')),
 ('host_id', dtype('int64')),
 ('host_since', dtype('<M8[ns]')),
 ('host_listings_count', dtype('float64'))]

In [0]:
updated_df['last_scraped'].dtype

NameError: ignored

Because these are the dtypes that we want to keep, we can then update our dataframe with them.

In [0]:
updated_df = converted_nums_df.astype(dtypes)

Now we can see that we have 38 columns in the correct format.  Not to bad.

In [0]:
updated_df.select_dtypes(exclude = 'object').shape

(8000, 38)

In [0]:
updated_df.to_csv('./nums_and_dates_ten_k.csv')

### Summary

In this lesson, we worked through our dataset to convert columns to be of type datetime and numeric columns.  With datetimes, we used apply to loop through our almost date columns, and convert each to a datetime, and then updated our dataframe.  

With the numeric columns we again identified the almost numeric columns, then we wrote functions to coerce them into numbers, and integrated the functions into our FunctionTransformer to use with our DataFrameMapper.